# COGS 108 Final Project

## Important

- text box


## Overview

Read our project description in the proposal notebook

## Group Members: 
- A14178715
- A11475687
- A12594395
- A11729103

### The Project:

In [ ]:
# imports 
import numpy as np
import pandas as pd
import matplotlib as plt


In [ ]:
# read in the data, nasa_data.txt as csv


In [ ]:
# clean data


In [ ]:
# plot data using plt.something()
